In [1]:
"""
Exercise 01: Supervisor System - Starter Code
=============================================
Build a supervisor coordinating research and writing sub-agents.

LEARNING GOALS:
- Create sub-agents with create_agent
- Wrap sub-agents as tools
- Understand supervisor routing via tool calling
"""

import os
from dotenv import load_dotenv

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

load_dotenv()

model = init_chat_model("gpt-4o-mini", model_provider="openai")


# =============================================================================
# TODO 1: Create Sub-Agents
# =============================================================================
# Create two specialized agents:
#
# Research Agent:
# - Name: "research_agent"
# - System prompt explaining it gathers comprehensive information
# - Include CRITICAL note: must include ALL findings in response
#
# Writing Agent:
# - Name: "writing_agent"  
# - System prompt explaining it creates polished content
# - Include CRITICAL note: must include COMPLETE content in response
#
# Why the CRITICAL notes? Sub-agents only return their final message!
#
# EXPERIMENT: How does the system prompt affect output quality?
# EXPERIMENT: Try adding specific formatting instructions
# =============================================================================

research_agent = create_agent(
    name="research_agent",
    model=model,
    tools=[],
    system_prompt="""You are a research specialist. Your role is to gather comprehensive information about topics.
    
CRITICAL: You must include ALL findings in your response. Sub-agents only return their final message, so ensure your response contains complete research results."""
)

writing_agent = create_agent(
    name="writing_agent",
    model=model,
    tools=[],
    system_prompt="""You are a writing specialist. Your role is to create polished, well-structured content based on research provided to you.
    
CRITICAL: You must include COMPLETE content in your response. Sub-agents only return their final message, so ensure your response contains the full written content."""
)



/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/Recature-Homework/Revature-Cognizant-Training-Exercises/January/venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:

# =============================================================================
# TODO 2: Create Tool Wrappers
# =============================================================================
# Wrap each sub-agent as a tool for the supervisor to call.
#
# research_topic tool:
# - Takes topic as parameter
# - Invokes research_agent with the topic
# - Extracts and returns the content from the result
#
# write_content tool:
# - Takes topic and research as parameters
# - Invokes writing_agent with both
# - Extracts and returns the content
#
# EXPERIMENT: What happens if you change the tool descriptions?
# EXPERIMENT: Add a "depth" parameter (brief vs comprehensive)
# =============================================================================

@tool
def research_topic(topic: str) -> str:
    """Research a topic thoroughly. Use this first to gather information."""
    result = research_agent.invoke({
        "messages": [{"role": "user", "content": f"Research the following topic thoroughly: {topic}"}]
    })
    return result["messages"][-1].content


@tool
def write_content(topic: str, research: str) -> str:
    """Write polished content based on research. Use after researching."""
    result = writing_agent.invoke({
        "messages": [{"role": "user", "content": f"Write polished content about: {topic}\n\nResearch findings:\n{research}"}]
    })
    return result["messages"][-1].content


In [3]:

# =============================================================================
# TODO 3: Create Supervisor Agent
# =============================================================================
# Create the supervisor that:
# - Has both tools available
# - Has a system prompt explaining the workflow:
#   1. First research
#   2. Then write based on research
# - Presents final content to user
#
# EXPERIMENT: Does the supervisor always follow the right order?
# EXPERIMENT: What prompts cause it to skip research?
# =============================================================================

agent = create_agent(
    model=model,
    tools=[research_topic, write_content],
    system_prompt="""You are a supervisor agent coordinating research and writing tasks.

Your workflow should be:
1. First, use the research_topic tool to gather comprehensive information about the topic
2. Then, use the write_content tool with both the topic and the research findings to create polished content
3. Present the final written content to the user

Always follow this order: research first, then write based on the research findings.""",
    name="supervisor_agent"
)

In [4]:
# =============================================================================
# Testing the Supervisor Agent
# =============================================================================

test_prompt = "Write a blog post about TypeScript benefits"

print(f"Test prompt: {test_prompt}\n")
print("=" * 80)
print()

result = agent.invoke({
    "messages": [{"role": "user", "content": test_prompt}]
})

print("Response:")
print("-" * 80)
print(result["messages"][-1].content)
print()
print("=" * 80)
print("\nCheck LangSmith to see the tool call sequence!")

Test prompt: Write a blog post about TypeScript benefits


Response:
--------------------------------------------------------------------------------
Here's a blog post about the benefits of using TypeScript:

---

### The Benefits of Using TypeScript

TypeScript, an open-source programming language developed by Microsoft, enhances the development process by serving as a superset of JavaScript. By introducing static typing and other valuable features, TypeScript empowers developers to write more robust, maintainable, and scalable code. Below, we explore the key benefits of using TypeScript in modern web development.

#### 1. Static Typing

One of the standout features of TypeScript is its ability to enforce static typing. This allows developers to specify types for variables, function parameters, and return values. As a result, many errors that typically surface at runtime can be detected during development. Consequently, developers can increase code reliability and significantly reduc